
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [ ]:
# python imports
import re
import json
from os.path import join
import cPickle
import numpy as np
import math
import itertools

# third party imports\
import pandas
import escher
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import (ribosome, tRNA_charging, transcription, translocation, 
                     chaperones, DNA_replication, translation)
from ecolime.characterization.biomass_allocation import get_membrane_composition, get_membrane_protein

from cobrame import *
from cobrame.util import building, dogma
from cobrame.util.mass import compute_RNA_mass
from cobrame.solve.algorithms import binary_search, fva, solve_at_growth_rate
from cobrame.solve.symbolic import compile_expressions

# Create Model Object and populate its global info
This includes important parameters that are used to calculate coupling constraints
as well as organism-specific information such as peptide processing types

In [ ]:
# Define Models
ijo = cobra.io.load_json_model('iJO1366.json')
me = MEmodel('iJO1366-ME')

# "Translational capacity" of organism
me.global_info['kt'] =  4.5 #(in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] =  0.087 #scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] =  1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1700. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000. #109. / 1000. # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000. # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000. # in kDA
me.global_info['f_tRNA'] = .12

me.global_info['RNA_polymerase'] = {'CPLX0-221', 'RNAPE-CPLX', 'CPLX0-222', 'RNAP32-CPLX',
                                    'RNAP54-CPLX', 'RNAP70-CPLX', 'RNAPS-CPLX'}

#me.global_info['c_ribo'] = me.global_info['m_rr'] / me.global_info['f_rRNA'] / me.global_info['m_aa']
#me.global_info['k_RNAP'] = mu * me.global_info['c_ribo']

# Translation associated global information
me.global_info["translation_terminators"] = translation.translation_stop_dict
me.global_info["met_start_codons"] = {"AUG", "GUG", "UUG", "AUU", "CUG"}
me.global_info["peptide_processing_subreactions"] = {"peptide_deformylase_processing": 1,
                                                     "peptide_chain_release": 1,
                                                     "ribosome_recycler": 1}
me.global_info["translation_elongation_subreactions"] = ['FusA_mono_elongation', 'Tuf_gtp_regeneration']
me.global_info['translation_start_subreactions'] = ['fmet_addition_at_START']

# This may be unnecessary 
# Transcription associated global information
me.global_info["transcription_initiation_length"] = 16  # nucleotides

me.global_info['translocation_multipliers'] = defaultdict(dict)
for enzyme, value in translocation.multipliers.items():
    me.global_info['translocation_multipliers'][enzyme] = value

# Folding Properties
me.global_info['temperature'] = 37
me.global_info['propensity_scaling'] = .45

# DNA Replication Parameters
me.global_info['GC_fraction'] = 0.507896997096

## Begin by loading metabolites and build Metabolic reactions

In [ ]:
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Suf"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# If lower_bound open, model feeds G6P into EDD
me.reactions.EX_pqq_e.lower_bound = 0
me.reactions.EX_pqq_e.upper_bound = 0

In [ ]:
# Used for mass balance checks
df = pandas.read_table('../ecolime/modification.txt', names=['mod', 'formula','na'])
df = df.drop('na', axis=1).set_index('mod').dropna(how='any')
me.global_info['modification_formulas'] = df.T.to_dict()

In [ ]:
essential_list = [
    'LI_c',
    'cs_e',
    'tl_c', # added in old ME but not metioned in supplement tables
#    'Oxidized_c',
#    'palmitate_c', # TODO FIX. this should not be required
    'cu_c', # needed for NDH activity. TODO fix this
    'C10H8O5_c', 'C9H9O4_c', # for tRNA modifications
    'NiFeCoCN2_c', #'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model, fix how cs is added to

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

In [ ]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)

building.build_reactions_from_genbank(me, gb_filename, TU_df, verbose=False,
                                      frameshift_dict=translation.frameshift_dict,
                                     tRNA_to_codon=translation.tRNA_to_codon)

### Add tRNA mods to tRNA charging reactions (iOL uses keffs of 65. for all modifications)

In [ ]:
for mod, components in get_tRNA_modification_procedures().items():
    tRNA_mod = ModificationData(mod, me)
    tRNA_mod.enzyme = components['machines']
    tRNA_mod.stoichiometry = components['metabolites']
    tRNA_mod.keff = 65. # iOL uses 65 for all tRNA mods
    if 'carriers' in components.keys():
        for carrier, stoich in components['carriers'].items():
            if stoich < 0:
                tRNA_mod.enzyme += [carrier]
            tRNA_mod.stoichiometry[carrier] = stoich
            
# tRNA_modifications = {tRNA_id: {modifications: count}}
tRNA_modifications=get_tRNA_modification_targets()
for tRNA in tRNA_modifications:
    for data in me.tRNA_data.query(tRNA):
        data.modifications = tRNA_modifications[tRNA]

## Add translation related subreactions

In [ ]:
# add the translation subreaction data objects to model
translation.add_translation_subreactions_to_model(me)

# add translation subreaction data to reactions
methionine_cleaved=translation.methionine_cleaved
folding_dict=translation.folding_dict
terminator_dict=translation.translation_stop_dict

for data in me.translation_data:
    data.term_enzyme = terminator_dict.get(
            data.last_codon)
    
    locus_id = data.id
    if locus_id in methionine_cleaved:
        data.subreactions['N_terminal_methionine_cleavage'] = 1

    for folding_type in folding_dict:
        if locus_id in folding_dict[folding_type]:
            data.subreactions[folding_type] = 1

    for subreaction, value in data.elongation_subreactions.items():
        data.subreactions[subreaction] = value

    for subreaction in data.translation_start_subreactions:
        data.subreactions[subreaction] = 1

    for subreaction in data.translation_termination_subreactions:
        data.subreactions[subreaction] = 1

    # add organism specific subreactions associated with peptide processing
    global_info = me.global_info
    for subrxn, value in global_info['peptide_processing_subreactions'].items():
        data.subreactions[subrxn] = value

In [ ]:
# Changed verbose=False to squash complex creation output
ribosome.add_ribosome(me, verbose=False)

## Add transcription related subreactions

In [ ]:
for met in me.global_info['RNA_polymerase']:
    RNAP_obj = RNAP(met)
    me.add_metabolites(RNAP_obj)

In [ ]:
for subreaction in transcription.transcription_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    enzymes = transcription.transcription_subreactions[subreaction]['enzymes']
    subreaction_data.stoichiometry = transcription.transcription_subreactions[subreaction]['stoich']
    subreaction_data.enzyme = enzymes

In [ ]:
data = ComplexData('RNA_degradosome', me)
for subunit, value in ecoli_k12.RNA_degradosome.items():
    data.stoichiometry[subunit] = value
data.create_complex_formation(verbose=False)

data = ModificationData('RNA_degradation_machine', me)
data.enzyme = 'RNA_degradosome'

data = ModificationData('RNA_degradation_atp_requirement', me)
# .25 water equivaltent for atp hydrolysis and 1 h20 equivalent for to 
# restore OH group in nucleotide monophosphate during RNA hydrolysis
data.stoichiometry = {'atp_c': -.25, 'h2o_c': -1.25, 'adp_c': .25,
                      'pi_c': .25, 'h_c': 1.25}

In [ ]:
for subreaction in me.subreaction_data:
    if type(subreaction.enzyme) == list:
        for enzyme in subreaction.enzyme:
            enzyme_met = Complex(enzyme)
            me.add_metabolites([enzyme_met])
    elif type(subreaction.enzyme) == str:
        enzyme_met = Complex(subreaction.enzyme)
        me.add_metabolites([enzyme_met])

## Add transcription

In [ ]:
sigma_to_RNAP_dict = transcription.sigma_factor_complex_to_rna_polymerase_dict
# Moved from buidling to make that function organism unspecific
for TU_id in TU_df.index:
    transcription_data = me.transcription_data.get_by_id(TU_id)
    rho_dependent = TU_df.rho_dependent[TU_id]
    sigma = TU_df.sigma[TU_id]
    RNA_polymerase = sigma_to_RNAP_dict[sigma]
    transcription_data.RNA_polymerase = RNA_polymerase
    transcription_data.rho_dependent = rho_dependent

### Make RNA splicing machinery reactions

In [ ]:
ecolime.transcription.add_RNA_splicing(me)
transcription.add_RNA_polymerase_complexes(me, verbose=False)
# moved from building (E. coli specific)
# add transcription machinery based on contained RNA types
# info added in buidling however
for transcription_data in me.transcription_data:
    if transcription_data.rho_dependent:
        rho = 'dependent'
    else:
        rho = 'independent'
    if transcription_data.codes_stable_rna:
        stable = 'stable'
    else:
        stable = 'normal'

    transcription_data.subreactions['Transcription_%s_rho_%s' % (stable,
                                                                 rho)] = 1

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [ ]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

## Add in complex Formation with modifications

In [ ]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
complex_stoichiometry_dict, complex_modification_dict = get_complex_composition(rna_components)
building.add_model_complexes(me, complex_stoichiometry_dict, complex_modification_dict)

In [ ]:
# two different reactions can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.stoichiometry = cplx_data.stoichiometry
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications.copy()
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [ ]:
# chaperones          
bmocogdp_chaperones = {'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp
for chaperone in set(bmocogdp_chaperones.values()):
    new_mod = ModificationData('mod_bmocogdp_c_' + chaperone, me)
    new_mod.enzyme = chaperone
    new_mod.stoichiometry = {'bmocogdp_c': -1}

for cplx_data in me.modification_data.get_by_id('mod_bmocogdp_c').get_complex_data():
    cplx_id = cplx_data.id.split('_mod')[0]
    if cplx_id in bmocogdp_chaperones:
        cplx_data.modifications['mod_bmocogdp_c_' + bmocogdp_chaperones[cplx_id]] = \
            cplx_data.modifications.pop('mod_bmocogdp_c')

 - Read "The CO and CN− ligands to the active site Fe in [NiFe]-hydrogenase of Escherichia coli have different metabolic origins" for insight into the metabolic origin of the NiFeCoCN2_c group

In [ ]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

for i in ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra']:
    me.add_metabolites([Complex(i)])

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

In [ ]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

In [ ]:
# Add known specific chaperone
fes_chaperones = {'CPLX0-1762':'G6712-MONOMER'} # FE-S modification
for chaperone in set(fes_chaperones.values()):
    new_mod = ModificationData('mod_2fe2s_c_' + chaperone, me)
    new_mod.enzyme = chaperone
    new_mod.stoichiometry = {'2fe2s_c': -1}
for cplx_data in me.modification_data.get_by_id('mod_2fe2s_c').get_complex_data():
    cplx_id = cplx_data.id.split('_mod')[0]
    if cplx_id in fes_chaperones:
        cplx_data.modifications['mod_2fe2s_c_' + fes_chaperones[cplx_id]] = \
            cplx_data.modifications.pop('mod_2fe2s_c')

##Build all complex formation reactions

In [ ]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

## Add dummy reaction to model and unmodeled_protein_fraction
 - Includes the transcription, translation, complex_formation, and metabolic reaction

In [ ]:
codons = pandas.read_csv(join(ecoli_files_dir, "codon_usage.csv"), index_col=0)

In [ ]:
seq = "ATG"
for codon, row in codons.iterrows():
    if row.amino_acid == "Stop":
        continue
    seq += codon * int(row.per_1000 // 3)  # want roughly 300 aa
# get the most used stop codon
seq += codons[codons.amino_acid == "Stop"].sort_values("per_1000").index[-1]
building.add_dummy_reactions(me, seq, update=True)

rxn = SummaryVariable('dummy_protein_to_mass')
me.add_reaction(rxn)
me.add_metabolites([Constraint('dummy_protein_biomass')])
mass = me.metabolites.protein_dummy.mass
rxn.add_metabolites({'protein_biomass': -mass, 'protein_dummy': -1,
                     'dummy_protein_biomass': mass})

## Associate Complexes with Reactions

In [ ]:
# associate reaction id with the old ME complex id (including modifications)
rxn_to_cplx_dict = get_reaction_to_complex()
rxn_info = get_reaction_info_frame()
building.add_reactions_from_stoichiometric_data(me, rxn_to_cplx_dict, rxn_info, update=True)

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [ ]:
for generic, components in ecoli_k12.generic_dict.items():
    GenericData(generic, me, components).create_reactions()

In [ ]:
apoACP = me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p')
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    if apoACP in reaction.reactants:
        reaction.complex_dilution_set = {apoACP.id}
        reaction.update()
me.reactions.acp_lipoate_synthase_FWD_SPONT.complex_dilution_set = {'CPLX0-782_mod_2:4fe4s'}
me.reactions.MOADSUx1_FWD_CPLX_dummy.complex_dilution_set = {'EG11597-MONOMER_mod_amp'}

## Add in translocation

In [ ]:
for pathway, info in translocation.pathway.items():
    if 'alt' not in pathway:
        transloc_data = TranslocationData(pathway + '_translocation', me)
    else:
        transloc_data = TranslocationData(pathway.replace('_alt', '_translocation_alt'), me)
    transloc_data.enzyme_dict = info['enzymes']
    transloc_data.keff = info['keff']
    transloc_data.length_dependent_energy = info['length_dependent_energy']
    transloc_data.stoichiometry = info['stoichiometry']

In [ ]:
transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
translocation.add_translocation_pathways(me, transloc, membrane_constraints = False)

In [ ]:
# Update stoichiometry of membrane complexes
# new_stoich = {complex_id: protein_w_compartment}
new_stoich = defaultdict(dict)
for cplx, row in transloc.set_index('Complex').iterrows():
    protein = row.Protein.split('(')[0] + '_' + row.Protein_compartment
    value = row.Protein.split('(')[1][:-1].split(':')[0]
    new_stoich[cplx]['protein_' + protein] = float(value)

In [ ]:
for cplx, stoich in new_stoich.items():
    complex_data = me.complex_data.get_by_id(cplx)
    for met, value in stoich.items():
        complex_data.stoichiometry.pop(met[0:13])
        complex_data.stoichiometry[met] = value
        complex_data.formation.update()
    # Complex ids in protein compartment file doesn't include mods
    # Some have multiple alternative modifications so must loop through these
    for complex_data in me.complex_data.query(cplx + '_mod_'):
        for met, value in stoich.items():
            complex_data.stoichiometry.pop(met[0:13])
            complex_data.stoichiometry[met] = value
            complex_data.formation.update()

## Add lipoprotein formation

In [ ]:
compartment_dict = {}
for prot, compartment in transloc.set_index('Protein').Protein_compartment.to_dict().items():
    compartment_dict[prot.split('(')[0]] = compartment

#### Add lipid modification ModificationData

In [ ]:
lipid_modifications = translocation.lipid_modifications

for lipid in lipid_modifications:
    data = ModificationData('mod_' + lipid, me)
    data.stoichiometry = {lipid: -1, 'g3p_c': 1}
    data.enzyme = ['Lgt_MONOMER', 'LspA_MONOMER']

data = ModificationData('mod2_pg160_p', me)
data.stoichiometry = {'pg160_p': -1, '2agpg160_p': 1}
data.enzyme = 'EG10168-MONOMER'

data = ModificationData('mod2_pe160_p', me)
data.stoichiometry = {'pe160_p': -1, '2agpe160_p': 1}
data.enzyme = 'EG10168-MONOMER'
    
translocation.add_lipoprotein_formation(me, compartment_dict, membrane_constraints=False)

#### Correct complex formation IDs if they contain lipoproteins

In [ ]:
for gene in translocation.lipoprotein_precursors.values():
    compartment = compartment_dict.get(gene)
    for rxn in me.metabolites.get_by_id('protein_' + gene + '_' + compartment).reactions:
        if isinstance(rxn, ComplexFormation):
            data = me.complex_data.get_by_id(rxn.complex_data_id)
            value = data.stoichiometry.pop('protein_' + gene + '_' + compartment)
            data.stoichiometry['protein_' + gene + '_lipoprotein' + '_' + compartment] = value
            rxn.update()

## Add Biomass Component Demands

In [ ]:
rxn = SummaryVariable('core_structural_demand_brauns')
met = Constraint('component_demand_biomass')
me.add_metabolites([met])
met1 = me.metabolites.get_by_id('murein5px4p_p')
met1_mass = met1.formula_weight / 1000.
met2 = me.metabolites.get_by_id('protein_b1677_lipoprotein_Outer_Membrane')
met2_mass = met2.formula_weight / 1000.
me.add_reaction(rxn)
rxn.add_metabolites({met1 : -0.013894, met2: -0.003597, 'component_demand_biomass':(0.013894 * met1_mass + 
                                                                                    0.003597 * met2_mass)},
                    combine=False)
rxn.lower_bound = mu
rxn.upper_bound = mu

In [ ]:
# Demand for lipid components
lipid_components = {}
requirement = 0.01945  # in mmol/gDW
met = me.metabolites.get_by_id('kdo2lipid4_e')
component_mass = met.formula_weight / 1000.
rxn = MetabolicReaction('Demand_' + met.id)
me.add_reaction(rxn)

data = StoichiometricData(rxn.id,me)
data.stoichiometry.update({met.id : -1.*requirement,
                           'component_demand_biomass': component_mass * requirement})
data.lower_bound = mu
data.upper_bound = mu
rxn.stoichiometric_data = data
rxn.update()

## Add lipid demands

In [ ]:
lipid = re.compile('(\d\d\d)(_)(.)')
lipid_demand = {}
for key, value in ijo.reactions.Ec_biomass_iJO1366_WT_53p95M.metabolites.items():
    if lipid.search(key.id):
        lipid_demand[key.id] = abs(value)
                
for met, requirement in lipid_demand.items():
    component_mass = me.metabolites.get_by_id(met).formula_weight / 1000.
    rxn = Reaction('Demand_' + met)
    me.add_reaction(rxn)
    rxn.add_metabolites({met: -1 * requirement,
                         'component_demand_biomass': component_mass * requirement})
    rxn.lower_bound = requirement
    rxn.upper_bound = 1000.
    


### Incorporate necessary biomass constituents

In [ ]:
# There are leftover components from the iJO biomass equation that either 
# 1) have no mechanistic function in the model (glycogen)
# 2) are cofactors that are regenerated (nad)
biomass_components = {
    "glycogen_c": -.023 / (me.metabolites.glycogen_c.formula_weight / 1000.),
    "2ohph_c": -0.000223,
    "nad_c": -.001831,
    "udcpdp_c": -5.5e-05,
    "coa_c": -0.000576,
    "ribflv_c": -0.000223,
    "fad_c": -0.000223,
    "mlthf_c": -0.000223,
    "thf_c": -0.000223,
    "10fthf_c": -0.000223
}


rxn = SummaryVariable('biomass_component_demand')
met = Constraint('component_demand_biomass')
me.add_reaction(rxn)
rxn.add_metabolites(biomass_components)
component_mass = sum(me.metabolites.get_by_id(c).formula_weight / 1000. * -v
                     for c, v in biomass_components.items())
rxn.lower_bound = mu
rxn.upper_bound = mu
me.reactions.biomass_component_demand.add_metabolites({met: component_mass})

rxn = SummaryVariable('biomass_component_dilution')
me.add_reaction(rxn)
rxn.add_metabolites({met: -1, me._biomass: 1})

## DNA replication

In [ ]:
dna_demand_stoich = DNA_replication.return_gr_dependent_dna_demand(me.global_info['GC_fraction'])

data = ComplexData('DNA_polymerase', me)
stoichiometry = DNA_replication.DNA_polymerase_stoichiometry
data.stoichiometry = {"protein_" + bnum: value for bnum, value in stoichiometry.items()}
data.create_complex_formation(verbose=False)

DNA_replication = SummaryVariable("DNA_replication")
me.add_reaction(DNA_replication)
DNA_replication.add_metabolites(dna_demand_stoich)
DNA_biomass = Constraint("DNA_biomass")
DNA_biomass.elements = {e: abs(v) for e, v in DNA_replication.check_mass_balance().items()}

dna_mw = 0
for met, value in me.reactions.DNA_replication.metabolites.items():
    if met.id != 'ppi_c':
        dna_mw -= value*util.mass.dna_mw_no_ppi[met.id.replace('_c','')] / 1000.
        
DNA_replication.add_metabolites({DNA_biomass: dna_mw})
DNA_replication.lower_bound = mu
DNA_replication.upper_bound = mu

# Comment out DNA_polymerase for now

#DNA_replication.add_metabolites({'DNA_polymerase': -1e-4})
me._DNA_biomass_dilution.lower_bound =0
me._DNA_biomass_dilution.upper_bound = 1000

## Attempt to set keffs

In [ ]:
keff_list = []
keffs = get_reaction_keffs(me, verbose=True)
for reaction_id, keff in keffs.items():
    if keff > 3000:
        keff = 3000.
    elif keff < .01:
        keff = .01
    keff_list.append(keff)
    me.reactions.get_by_id(reaction_id).keff = keff
    me.reactions.get_by_id(reaction_id).update()

In [ ]:
# Keffs that were not set in the above cell
me.subreaction_data.N_terminal_methionine_cleavage.keff = 1339.4233102860871
me.subreaction_data.peptide_deformylase_processing.keff = 1019.5963333345715
me.reactions.get_by_id('GLUTRR_FWD_CPLX0-3741').keff = 3000 # 3269.0108007383374
me.subreaction_data.fmet_addition_at_START.keff = 1540.4356849968603
me.subreaction_data.ribosome_recycler.keff = 1059.6910912619182
me.subreaction_data.UAG_PrfA_mono_mediated_termination.keff = 1721.7910609284945
me.subreaction_data.UGA_PrfB_mono_mediated_termination.keff = 1700.2966587695353
me.subreaction_data.UAA_generic_RF_mediated_termination.keff = 1753.4238515034572

In [ ]:
# These should be coupled to translation not ribosome formation per iOL1650
Inf_modifications = ['Translation_initiation_factor_InfA', 'Translation_gtp_initiation_factor_InfB',
                    'Translation_initiation_factor_InfC']
for sub in Inf_modifications:
    me.complex_data.ribosome.modifications[sub] = 0
    for data in me.translation_data:
        data.modifications[sub]= -1

## Clean up and update everything

In [ ]:
# RNA_dummy, TU_b3247, TU_b3705 do not have RNAP, this is set as the most common RNAP
for data in me.transcription_data:
    if len(data.RNA_polymerase) == 0:
        data.RNA_polymerase = 'RNAP70-CPLX'

## Model updates and corrections

### Include peripheral proteins to the membrane capacity constraint

In [ ]:
# Add reaction subsystems from iJO to model
for rxn in ijo.reactions:
    if rxn.id in me.stoichiometric_data:
        data = me.stoichiometric_data.get_by_id(rxn.id)
    else:
        continue
    for r in data.parent_reactions:
        r.subsystem = rxn.subsystem

In [ ]:
# Newly annotated modification gene (w/ high confidence)
mod = me.modification_data.get_by_id('mod_acetyl_c')
mod.enzyme = 'RimL_mono'
mod.stoichiometry = {'accoa_c':-1, 'coa_c':1}

# Correction from iML
me.reactions.get_by_id('DHPTDNRN_REV_DIHYDROPTERIREDUCT-CPLX_mod_fad').upper_bound = 0
me.reactions.get_by_id('DHPTDNR_REV_DIHYDROPTERIREDUCT-CPLX_mod_fad').upper_bound = 0

# cobalamin is not essential in e. coli and there is no evidence
# of this gene requiring this modification.
me.reactions.EX_cbl1_e.lower_bound = 0
me.complex_data.QueG_mono_mod_adocbl.modifications.pop('mod_adocbl_c')
me.complex_data.QueG_mono_mod_adocbl.formation.update()
print me.reactions.formation_QueG_mono_mod_adocbl.reaction

# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMpp']
for reaction in KO_list:
    data = me.stoichiometric_data.get_by_id(reaction)
    data.lower_bound = 0
    data.upper_bound = 0
    
# this RNAP/sigma factor should not be used to transcribe stable rna   
for rxn in me.metabolites.get_by_id('RNAP32-CPLX').reactions:
    if rxn.id != 'formation_RNAP32-CPLX' and rxn.transcription_data.codes_stable_rna:
        rxn.upper_bound = 0
        print rxn
        
me.reactions.NH4tpp_FWD_SPONT.remove_from_model()
me.reactions.NH4tpp_REV_SPONT.remove_from_model()

# I dont think this complex is correct
me.reactions.get_by_id('formation_ATPSYN-CPLX_EG10106-MONOMER').knock_out()

# add oxidative damage reaction to form 3fe4s 
# TODO this can be done better
fes_damage = cobra.Reaction('4fe4s_oxidation')
me.add_reaction(fes_damage)
fes_damage.reaction = '4fe4s_c -> fe2_c + 3fe4s_c'

In [ ]:
me.update()
me.prune()

In [ ]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.
me.reactions.EX_glc__D_e.lower_bound = -1000
me.reactions.EX_o2_e.lower_bound = -1000.
# we can now set the unmodeled protein fraction
me.unmodeled_protein_fraction = 0.35
me.ngam = 13.94
me.gam = 35  

In [ ]:
n_genes = len(me.metabolites.query(re.compile('RNA_b[0-9]')))
# 3 genes in iJL unused 
# 1 new gene (rimL) added to model above
print("number of genes in the model %d (%.2f%%)" % (n_genes, n_genes * 100. / (1678))) 

## Solve

In [ ]:
with open("prototype_67.pickle", "wb") as outfile:
    cPickle.dump(me, outfile)

In [ ]:
using_soplex=False
if using_soplex:
    binary_search(me, min_mu=.1, max_mu=.3, debug=False, mu_accuracy=1e-2)
else:
    from qminos.me1 import ME_NLP1
    me.reactions.EX_o2_e.lower_bound = -16.
    me.reactions.EX_glc__D_e.lower_bound = -1000.
    me.reactions.EX_glc__D_e.upper_bound = 1000.
    me.reactions.EX_ac_e.lower_bound = 0
    # The object containing solveME methods--composite that uses a ME model object 
    # Provide growth_key = 'mu' for minime models,
    me_nlp = ME_NLP1(me, growth_key='mu')
    # Use bisection for now (until the NLP formulation is worked out for the new prototype 44
    muopt, hs, xopt, cache = me_nlp.bisectmu(precision=1e-2, mumax=1.5)    
    # Access the solution that is saved in the original cobrame object
    sol = me.solution
    sol.f
    sol.x_dict

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux(solution=me.solution)
view.display_in_notebook()